In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyfbs

### Define parameters

In [ ]:
eosDD2 = pyfbs.PyEoStable("/media/data/Documents/PhD/B09/B09/EOS_tables/eos_HS_DD2_with_electrons.beta")
#eosCausal = pyfbs.PyCausalEoS(1e-10)

### Test single star integration

In [ ]:
mu = 5
lam = 0.
rho_c =  0.004
phi_c = 0.1

In [ ]:
myFBS = pyfbs.PyFermionBosonStar.FromParameters(eosDD2, mu, lambda_=lam, rho_0=rho_c, phi_0=phi_c)
myFBS.bisection(1., 10)
myFBS.get()
res = myFBS.evaluate_model()
myFBS.get()

In [ ]:
fig = plt.figure(figsize=(12,8)); ax = fig.gca()
myFBS.plot(ax)
r = res[:,0]
M_T = r /2. * (1. - 1./res[:,1]**2)
l, = ax.loglog(r, M_T, label='M_T')
ax.axhline(myFBS.get()['M_T'], linestyle='--', color=l.get_c())
ax.axvline(myFBS.get()['R_F'], linestyle='--')
ax.axvline(myFBS.get()['R_B_0'], linestyle='--')
ax.set_xscale('log'); ax.set_yscale('log')
ax.set_xlim(left=1e-3)
ax.set_ylim(bottom=1e-16, top=1e5)
ax.legend(); ax.grid()
print(r[-1])

In [ ]:
myFBS = pyfbs.PyFermionBosonStarTLN.FromFBS(myFBS)
myFBS.bisection_phi_1(1e-3 * phi_c, 1e5 * phi_c)
res = myFBS.evaluate_model()
myFBS.get()

In [ ]:
fig = plt.figure(figsize=(12,8)); ax = fig.gca()
myFBS.plot(ax)
r = res[:,0]
y = r * res[:,7]/res[:,6]
l, = ax.loglog(r, y, label='y')
ax.axhline(myFBS.get()['y_max'], linestyle='--', color=l.get_c())
ax.axvline(myFBS.get()['R_ext'], linestyle='--', color=l.get_c())
ax.set_xscale('log'); ax.set_yscale('log')
ax.set_xlim(left=1e-3)
ax.set_ylim(bottom=1e-8, top = 1e4)
ax.legend(); ax.grid()

### Compare to Fig 2. of https://arxiv.org/pdf/1606.03035.pdf

In [ ]:
mu = 1.
lam = 0.
rho_c = np.array([0.])
phi_c = np.geomspace(5e-4, 1e-1, 50)
pMR = pyfbs.PyMRcurve.from_rhophi_list(mu, lam, eosDD2, rho_c, phi_c, "")
tln_curve = pyfbs.PyMRcurve.calc_TLN_curve(pMR)

In [ ]:
M = np.array(  [fbs.get()['M_T'] for fbs in tln_curve])
Lam = np.array([fbs.get()['lambda_tidal'] for fbs in tln_curve])

In [ ]:
plt.plot(phi_c, Lam*M**5)
plt.grid()
plt.ylabel("$\lambda_tidal$"); plt.ylim(bottom=0, top=1200)
plt.xlabel("$\phi_c$"); plt.xlim(left=0., right=0.05)

In [ ]:
# look at a specific instance
fig = plt.figure(figsize=(12,8)); ax = fig.gca()
fbs = tln_curve[0]
int_res = fbs.evaluate_model()

fbs.plot(plt.gca())
r = int_res[:,0]

M_T = r /2. * (1. - 1./int_res[:,1]**2)
l, = ax.loglog(r, M_T, label='M_T')
ax.axhline(fbs.get()['M_T'], linestyle='--', color=l.get_c())

y = r * int_res[:,7]/int_res[:,6]
l, = ax.loglog(r,y, label='y')
ax.axvline(fbs.get()['R_ext'], linestyle='--', color=l.get_c())
ax.axhline(fbs.get()['y_max'], linestyle='--', color=l.get_c())

ax.set_xscale('log'); ax.set_yscale('log')
ax.set_xlim(left=1e-3)
ax.set_ylim(bottom=1e-12)
ax.legend(); ax.grid()
fbs.get()

### Compare to DD2 k2

In [ ]:
rho_c = np.geomspace(1e-4, 1e-2, 88)
phi_c = np.array([0.])
pMR = pyfbs.PyMRcurve.from_rhophi_list(mu, lam, eosDD2, rho_c, phi_c, "")
tln_curve = pyfbs.PyMRcurve.calc_TLN_curve(pMR)

In [ ]:
M = np.array([fbs.get()['M_T'] for fbs in tln_curve])
R = np.array([fbs.get()['R_F'] for fbs in tln_curve])
k2 = np.array([fbs.get()['k2'] for fbs in tln_curve])
C = M/R / 1.477

In [ ]:
plt.plot(C, k2)
plt.grid()
plt.xlabel("C"); #plt.ylim(bottom=0, top=1200)
plt.ylabel("$k_2$"); #plt.xlim(left=0., right=0.05)

## General Test Set

In [ ]:
rho_cs = np.geomspace(1e-4, 1e-2, 3)
phi_cs = np.geomspace(1e-5, 1e-1, 4)
mus = np.geomspace(1e-1, 1e1, 3)
lams = np.append([0.], np.geomspace(1e1, 1e3, 3))
filename = "data/testset.txt"
pMR = []
for rho_c in rho_cs:
    for phi_c in phi_cs:
        for mu in mus:
            for lam in lams:
                pMR.append(pyfbs.PyFermionBosonStar.FromParameters(eosDD2, mu, lambda_=lam, rho_0=rho_c, phi_0=phi_c))


In [ ]:
pMR = pyfbs.PyMRcurve.from_list(pMR)
tln_curve = pyfbs.PyMRcurve.calc_TLN_curve(pMR, filename)